Jing (Thomas) Zhang

## Problem 1

In [137]:
STATES = {-2, -1, 0, 1, 2}
ACTIONS = {"-1", "+1"}
GAMMA = 1

In [138]:
def Actions(s):
    if IsEnd(s):
        return {}
    return ACTIONS

In [139]:
def IsEnd(s):
    return s == 2 or s == -2

In [163]:
def get_next(s):
    return [s + 1, s - 1]

If you're in state s and choose -1:
- You have an 80% chance of reaching the state s−1.
- You have a 20% chance of reaching the state s+1.

If you're in state s and choose +1:
- You have a 70% chance of reaching the state s+1.
- You have a 30% chance of reaching the state s−1.

In [164]:
def T(s, a, s_next):
    if IsEnd(s):
        return 0
    if a == "-1":
        if s_next == s - 1:
            return 0.8
        if s_next == s + 1:
            return 0.2
    if a == "+1":
        if s_next == s + 1:
            return 0.7
        if s_next == s - 1:
            return 0.3

In [165]:
def Reward(s, a, s_next):
    if s_next == -2:
        return 20
    if s_next == 2:
        return 100
    return -5

$$Q_{\text{opt}}(s, a) = 
\sum_{s'} T(s, a, s')[\text{Reward(s, a, s')} + \gamma V_{\text{opt}}(s')] $$

In [196]:
def Q_opt(s, a, it):
    return sum(T(s, a, s_next) * (Reward(s, a, s_next) + GAMMA * logs[it][s_next])
              for s_next in get_next(s))

$$V_{\text{opt}} =  
\begin{cases}
0 & \mbox{if } \text{IsEnd}(s) \\
\text{max}_{a \in \text{Actions(s)}} Q_{\text{opt}}(s, a) &
\text{otherwise}
\end{cases}$$

In [197]:
def V_opt(s, it):
    if IsEnd(s):
        return 0
    return max(Q_opt(s, a, it) for a in Actions(s))

### Question a

Initialization

In [198]:
logs = {i: {s: 0 for s in STATES} for i in range(3)}

After iteration 0

In [199]:
for s, v in sorted(logs[0].items()):
    print("state {}:\t Vopt {}".format(s, v))

state -2:	 Vopt 0
state -1:	 Vopt 0
state 0:	 Vopt 0
state 1:	 Vopt 0
state 2:	 Vopt 0


Iteration 1

In [200]:
for s in STATES:
    logs[1][s] = V_opt(s, 0)

After iteration 1

In [201]:
for s, v in sorted(logs[1].items()):
    print("state {}:\t Vopt {}".format(s, v))

state -2:	 Vopt 0
state -1:	 Vopt 15.0
state 0:	 Vopt -5.0
state 1:	 Vopt 68.5
state 2:	 Vopt 0


Iteration 2

In [202]:
for s in STATES:
    logs[2][s] = V_opt(s, 1)

After Iteration 2

In [203]:
for s, v in sorted(logs[2].items()):
    print("state {}:\t Vopt {}".format(s, v))

state -2:	 Vopt 0
state -1:	 Vopt 14.0
state 0:	 Vopt 47.45
state 1:	 Vopt 67.0
state 2:	 Vopt 0


### Question b

$$\pi_{\text{opt}} = \text{argmax}_{a \in \text{Actions}(s)} Q_{\text{opt}}(s, a)$$

In [204]:
def pi_opt(s, it):
    return max([a for a in Actions(s)], key=lambda a: Q_opt(s, a, it))

$\pi_\text{opt}(s)$ for iteration 2 (Calculated using $V_\text{opt}$ in iteration 1)

In [206]:
for s in sorted(STATES):
    if IsEnd(s):
        continue
    print("optimal policy for state {} is {}".format(s, pi_opt(s, 1)))

optimal policy for state -1 is -1
optimal policy for state 0 is +1
optimal policy for state 1 is +1


## Problem 2

### Question b

By the definition of acyclic, there is no cycle in the graph, which means once we visited a state, we will never go back to it. So, we can simply use dynamic programming to go through all the (s, a, s') triples in one pass. The value after that iteration will be set correctly.

### Question c

$$
\begin{split}
Q_{\text{opt}}'(s, a) 
&= \sum_{s'} T(s, a, s')[\text{Reward(s, a, s')} + \gamma V_{\text{opt}}(s')]\\
&= \sum_{s'} T(s, a, s')\gamma[(\frac{1}{\gamma}) \text{Reward(s, a, s')} + (1)V_{\text{opt}}(s')]\\
&= \sum_{s'} \gamma T(s, a, s')[\frac{1}{\gamma} \text{Reward(s, a, s')} + (1)V_{\text{opt}}(s')]\\
&= \sum_{s'} T'(s, a, s')[\text{Reward'(s, a, s')} + \gamma' V_{\text{opt}}(s')]
\end{split}$$

for all $s'$:
- $T'(s, a, s') = \gamma T(s, a, s')$
- $\text{Reward'(s, a, s')} = \frac{1}{\gamma} \text{Reward(s, a, s')}$

Since the above equation equals the old equation $Q_{\text{opt}}$, the remaining term for the end state $o$ must equal to 0:

$$\begin{split}
T'(s, a, o)[\text{Reward'(s, a, o)} + V_{\text{opt}}(o)] 
&= T'(s, a, o)[\text{Reward'(s, a, o)} + 0]\\
&= T'(s, a, o)\text{Reward'(s, a, o)}\\
&= 0
\end{split}$$

Since $T'(s, a, o)$ does not have to equal to 0, $\text{Reward'(s, a, o)} = 0$

$T'(s, a, o) = 1 - \gamma$ (sum to 1)

## Problem 4

### Question b

Console log of 4b helper

Small MDP
- ValueIteration: 5 iterations
- 27 total, 25 same, 2 different
- 7.40740740741% diff

Large MDP
- ValueIteration: 15 iterations
- 2745 total, 1574 same, 1171 different
- 42.6593806922% diff

Q Learning performed much better on small MDP than large MDP. This is probably due to the fact that it was unable to learn well in a much larger state space, and the identity feature extractor can't describe unexplored states (can't generalize). 

### Question c

- ValueIteration: 15 iterations
- 2745 total, 2040 same, 705 different
- 25.6830601093% diff

### Question d

- VI then Fixed: 8.47406666667
- Double Q-learning: 9.14346666667

Q-learning has higher expected rewards then FixedRL because it can adapt to the MDP with the new threshold. For the VI then FixedRL, its policies may be optimal for the original MDP but may not be optimal for the new threshold, and it can't adapt to the new one as it lacks the ability to learn.